In [1]:
#import library
import os
import numpy as np
from rdkit import Chem
from tqdm import tqdm
import sys
sys.path.append("..")
from data_process import spec
from data_process.spec_to_wordvector import spec_to_wordvector
import gensim
from rdkit.Chem.Descriptors import ExactMolWt


D:\ANACONDA\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
#NEIMS predicted EI-MS.sdf to spectra.mgf
f = 'collected_compounds_predicted_EI-MS.sdf' 
suppl = Chem.SDMolSupplier(f)
smiles=[]
spectrums=[]
for j in tqdm(range(len(suppl))):
    smi = Chem.MolToSmiles(suppl[j])
    smiles.append(smi)
    try:
        spec_string = suppl[j].GetProp('PREDICTED SPECTRUM')
        A=spec_string.split('\n')
        Y=[x.split(' ') for x in A]
        W=[[float(x[0]),float(x[1])] for x in Y]
        c=np.array(W)
        M=np.array([x[0] for x in c])
        I=np.array([x[1] for x in c])
        I /= max(I)
        if max(M)>1000:
            continue
        else:                
            spectrum = spec.Spectrum(mz=M,intensities=I,metadata={'smiles': smi})
            spectrums.append(spectrum)

    except  KeyError:
        continue
#spec.save_as_mgf(spectrums, 'predicted_spectra.mgf') 

100%|█████████████████████████████████████████████████████████████████████████| 131145/131145 [03:50<00:00, 569.68it/s]


In [3]:
#filter
def filter_molecules(all_smiles):
    a=['Br','I','Cl','S','O','P','N','H','F','C','Si']
    logp=[]
    masses=[]
    log_idx=[]
    masses_idx=[]
    mol_idx=[]
    atom_idx=[]
    smiles_idx=[]
    from rdkit.Chem.Descriptors import ExactMolWt
    from rdkit.Chem import Descriptors
    for i in tqdm(range(len(all_smiles))):
        if '.' in all_smiles[i]:
            smiles_idx.append(i)
        
        mol = Chem.MolFromSmiles(all_smiles[i])
        if mol is None:
            mol_idx.append(i)
            continue           
        else:
        
            atoms=[a.GetSymbol() for a in mol.GetAtoms()]
            ma=ExactMolWt(mol)
            masses.append(ma)
            if ma>1000:
                masses_idx.append(i)
            c=Descriptors.MolLogP(mol)
            logp.append(c)
            if c<-12 or c>24:
                log_idx.append(i)
            c=list(set(atoms))
            for j in c:
                if j not in a:
                    atom_idx.append(i)
                    break
                else:
                    pass
    from collections import defaultdict
    d = defaultdict(list)
    for k,va in [(v,i) for i,v in enumerate(all_smiles)]:
        d[k].append(va)
    c=list(d.keys())
    nosame_idx=[]
    for i in range(len(c)):
        s=d[c[i]]
        if len(s)>1:
            for j in range(len(s)-1):
                nosame_idx.append(s[j+1])
        else:
            pass
    ss=masses_idx+log_idx+mol_idx+atom_idx+nosame_idx+smiles_idx
    index=list(set(ss))
    return index
filter_index=filter_molecules(smiles)
filter_spectrums=[spectrums[i] for i in range(0,len(spectrums),1) if i not in filter_index]
spec.save_as_mgf(filter_spectrums, 'collected_compounds_predicted_spectra.mgf') 

100%|█████████████████████████████████████████████████████████████████████████| 131145/131145 [02:31<00:00, 867.01it/s]


In [4]:
#build SQLite library
mass=[]
smiles=[]
mzs=[]
intensitys=[]
ID=[]
for i in range(len(filter_spectrums)):
    smiles.append(filter_spectrums[i].metadata['smiles'])
    mass.append(round(ExactMolWt(Chem.MolFromSmiles(filter_spectrums[i].metadata['smiles'])),3))
    ID.append('compound_'+str(i))
    s=filter_spectrums[i].peaks.to_np
    mz=[]
    intensity=[]
    for j in range(len(s[:,0])):
        mz.append(int(s[:,0][j]))
        intensity.append(round(s[:,1][j],5))        
    mzs.append(mz)
    intensitys.append(intensity)
    
import json
import sqlite3
conn = sqlite3.connect('YOUROWN_LIBRARY.db')  
c = conn.cursor()
c.execute('''CREATE TABLE YOUROWN_LIBRARY
      (COMPID TEXT  NOT NULL,
      SMILES           TEXT    NOT NULL,
      EXACTMOLWT       BLOB NOT NULL,
      MZS        BLOB NOT NULL,
      INTENSITYS        BLOB NOT NULL);''')
conn.commit()
conn.close()
conn = sqlite3.connect('YOUROWN_LIBRARY.db')
c = conn.cursor()
for i in tqdm(range(len(ID))):
    E=ID[i]
    F=smiles[i]
    Y=round(mass[i],3)
    G1=json.dumps(mzs[i])
    G2=json.dumps(intensitys[i])
    
    #G=json.dumps(file[i])
    c.execute("INSERT INTO YOUROWN_LIBRARY (COMPID,SMILES,EXACTMOLWT,MZS,INTENSITYS) VALUES (?,?,?,?,?)", (E,F,Y,G1,G2))
conn.commit()
conn.close()


100%|████████████████████████████████████████████████████████████████████████| 128171/128171 [00:27<00:00, 4707.48it/s]


In [5]:
#spectra.mgf to word2vec embeddings vectors
spectrums=filter_spectrums
print(len(spectrums))
model_file ="references _word2vec.model"
        # Load pretrained model (here dummy model)
model = gensim.models.Word2Vec.load(model_file)
spectovec = spec_to_wordvector(model=model, intensity_weighting_power=0.5)
word2vectors=[]
word_smiles=[]
spectrums = [s for s in spectrums if s is not None]
for i in tqdm(range(len(spectrums))):
    spectrum_in = spec.SpectrumDocument(spectrums[i], n_decimals=0)
    vetors=spectovec._calculate_embedding(spectrum_in)
    word_smiles.append(spectrum_in.metadata['smiles'])
    word2vectors.append(vetors)
np.save("collected_compounds_smiles.npy", word_smiles)
from scipy.sparse import csr_matrix, save_npz
word_vec=csr_matrix(np.array(word2vectors))
save_npz('collected_compounds_word_embeddings.npz', word_vec)

128171


100%|█████████████████████████████████████████████████████████████████████████| 128171/128171 [04:56<00:00, 432.96it/s]


In [6]:
#build HNSW index
from scipy.sparse import load_npz
import time
import hnswlib
import pickle
xb= load_npz('collected_compounds_word_embeddings.npz').todense().astype('float32')
xb_len =  np.linalg.norm(xb, axis=1, keepdims=True)
xb = xb/xb_len
dim = 500
num_elements =len(xb)
ids = np.arange(num_elements)
# Declaring index
p = hnswlib.Index(space = 'l2', dim = dim) # possible options are l2, cosine or ip
# Initializing index - the maximum number of elements should be known beforehand
p.init_index(max_elements = num_elements, ef_construction = 800, M = 64)
import time
start_time=time.time()*1000
# Element insertion 
p.add_items(xb, ids)
# Controlling the recall by setting ef:
p.set_ef(300) # ef should always be > k   ##
end_time=time.time()*1000
print('add_time %.4f'%((end_time-start_time)/100))


start_time=time.time()*1000
p.save_index('references_index.bin')
end_time=time.time()*1000
print('saveindex_time %.4f'%((end_time-start_time)/100))
# Index objects support pickling
p_copy = pickle.loads(pickle.dumps(p)) # creates a copy of index p using pickle round-trip
### Index parameters are exposed as class properties:
print(f"Parameters passed to constructor:  space={p_copy.space}, dim={p_copy.dim}") 
print(f"Index construction: M={p_copy.M}, ef_construction={p_copy.ef_construction}")
print(f"Index size is {p_copy.element_count} and index capacity is {p_copy.max_elements}")


add_time 915.9868
saveindex_time 6.4382
Parameters passed to constructor:  space=l2, dim=500
Index construction: M=64, ef_construction=800
Index size is 128171 and index capacity is 128171
